In [1]:
import os
os.chdir('../')

In [2]:
os.getcwd()

'/Users/abishekkumaryadav/DataScience/computer_vision/BoneFracture_classification'

In [3]:
# !pip install tensorflow==2.15.0

In [4]:
# daghub
# This is for terminal
""" 
MLFLOW_TRACKING_URI=https://dagshub.com/Abhikkumar619/BoneFracture_classification.mlflow \
MLFLOW_TRACKING_USERNAME=Abhikkumar619 \
MLFLOW_TRACKING_PASSWORD=b515b18fe70cac23bd1c8591a7c54e188845b00c \
python script.py
"""

' \nMLFLOW_TRACKING_URI=https://dagshub.com/Abhikkumar619/BoneFracture_classification.mlflow MLFLOW_TRACKING_USERNAME=Abhikkumar619 MLFLOW_TRACKING_PASSWORD=b515b18fe70cac23bd1c8591a7c54e188845b00c python script.py\n'

In [5]:
# This is for juypter notebook
os.environ['MLFLOW_TRACKING_URI']="https://dagshub.com/Abhikkumar619/BoneFracture_classification.mlflow"
os.environ['MLFLOW_TRACKING_USERNAME']="Abhikkumar619"
os.environ['MLFLOW_TRACKING_PASSWORD']="b515b18fe70cac23bd1c8591a7c54e188845b00c"

In [6]:
import tensorflow as tf
import keras

In [7]:
model=tf.keras.models.load_model("artifacts/training/best_model.h5")

In [8]:
from dataclasses import dataclass
from pathlib import Path

In [9]:
# pip install --upgrade tensorflow

In [10]:
@dataclass(frozen=True)
class EvaluationConfig: 
    path_of_model: Path
    training_data: Path
    all_parmas: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int
    train_data_path: Path
    valid_data_path: Path
    

In [11]:
from src.bone_classifier.constants import *
from src.bone_classifier import log
from src.bone_classifier.utils.common import read_yaml, create_directories, save_json

In [12]:
class configurationManager: 
    def __init__(self, config_file_path=CONFIG_FILE_PATH, 
                 params_file_path=PARAMS_FILE_PATH) -> None:
        self.config=read_yaml(config_file_path)
        self.parmas=read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self)-> EvaluationConfig: 
        config=self.config.model_evaluation
        params=self.parmas

        get_model_evaluation=EvaluationConfig(
            path_of_model=config.path_of_model,
            training_data=config.training_data,
            mlflow_uri="https://dagshub.com/Abhikkumar619/BoneFracture_classification.mlflow",
            all_parmas=params,
            params_batch_size=params.BATCH_SIZE,
            params_image_size=params.IMAGE_SIZE,
            train_data_path=config.train_data_path,
            valid_data_path=config.valid_data_path
            )
        return get_model_evaluation

In [13]:
import mlflow
import mlflow.keras
from urllib.parse import urlparse
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras

import mlflow
from tensorflow import keras

In [14]:
class Evaluation: 
    def __init__(self, config: EvaluationConfig) -> None:
        self.config=config
        
    def train_valid_generator(self):
        
        train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input)
        validation_datagen=ImageDataGenerator(preprocessing_function=preprocess_input)

        self.train_generator= train_datagen.flow_from_directory(
            self.config.train_data_path,
            target_size=(224,224),
            batch_size=self.config.params_batch_size
            
        )
        
        self.valid_generator=validation_datagen.flow_from_directory(
            self.config.valid_data_path,
            target_size=(224,224),
            batch_size=self.config.params_batch_size
        )
    @staticmethod
    def load_models(path: Path): 
        return tf.keras.models.load_model(path)
    
    def save_score(self): 
        score={'loss': self.score[0], "accuracy": self.score[1]}
        log.info(f"Score: {score}")
        save_json(path=Path("scores.json"), data=score)

    def evaluation(self): 
        self.model=self.load_models(self.config.path_of_model)
        self.train_valid_generator()
        self.score=model.evaluate(self.valid_generator)
        log.info(f"Evaluation score: {self.score}")
        self.save_score()

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store=urlparse(mlflow.get_tracking_uri()).scheme

        log.info(f"tracking url type store { tracking_url_type_store}")

        with mlflow.start_run():
            mlflow.log_params(self.config.all_parmas)
            mlflow.log_metrics(
                {"loss":  self.score[0], "accuracy": self.score[1]}
            )
            if tracking_url_type_store != 'file':
                mlflow.keras.log_model(self.model, artifact_path="model", registered_model_name="inception_v3")
            else:
                mlflow.keras.log_model(self.model, artifact_path="model")


In [15]:
config=configurationManager()
evaluation_config=config.get_evaluation_config()
evaluate=Evaluation(evaluation_config)
evaluate.train_valid_generator()
evaluate.evaluation()
evaluate.log_into_mlflow()


[2024-03-16 23:07:21,844 : INFO : common : yaml file : config/config.yaml loaded sucessfully]
[2024-03-16 23:07:21,845 : INFO : common : yaml file : params.yaml loaded sucessfully]
[2024-03-16 23:07:21,846 : INFO : common : Created directories at : artifacts]
Found 587 images belonging to 5 classes.
Found 73 images belonging to 5 classes.
Found 587 images belonging to 5 classes.
Found 73 images belonging to 5 classes.


2024-03-16 23:07:23.043402: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


10/10 [==============================] - 2s 116ms/step - loss: 2.0969 - accuracy: 0.5479
[2024-03-16 23:07:24,635 : INFO : 2422622669 : Evaluation score: [2.0969204902648926, 0.5479452013969421]]
[2024-03-16 23:07:24,635 : INFO : 2422622669 : Score: {'loss': 2.0969204902648926, 'accuracy': 0.5479452013969421}]
[2024-03-16 23:07:24,636 : INFO : 2422622669 : tracking url type store https]


2024/03/16 23:07:26 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /var/folders/4c/smh5_57j6lxd0dns3y7skny80000gn/T/tmpqmo6djef/model/data/model/assets
[2024-03-16 23:07:33,427 : INFO : builder_impl : Assets written to: /var/folders/4c/smh5_57j6lxd0dns3y7skny80000gn/T/tmpqmo6djef/model/data/model/assets]


/Users/abishekkumaryadav/DataScience/computer_vision/BoneFracture_classification/bone_env/lib/python3.8/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'inception_v3' already exists. Creating a new version of this model...
2024/03/16 23:09:55 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: inception_v3, version 4
Created version '4' of model 'inception_v3'.


In [16]:
import tensorflow as tf
print(tf.__version__)

2.13.0-rc0
